In [1]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val input = Path("day8.txt").readLines()

val exampleInput = listOf(
    "RL",
    "",
    "AAA = (BBB, CCC)",
    "BBB = (DDD, EEE)",
    "CCC = (ZZZ, GGG)",
    "DDD = (DDD, DDD)",
    "EEE = (EEE, EEE)",
    "GGG = (GGG, GGG)",
    "ZZZ = (ZZZ, ZZZ)",
)

# Part 1

In [2]:
enum class Direction { Right, Left }

fun parseDirections(input: List<String>): List<Direction> {
    return input[0].map {
        when (it) {
            'R' -> Direction.Right
            'L' -> Direction.Left
            else -> throw IllegalArgumentException("invalid direction $it")
        }
    }
}

typealias Position = String

fun parseInstructions(input: List<String>): Map<Position, Pair<Position, Position>> {
    return input.drop(2)
        .map {
            val (pos1, pos2, pos3) = it.split(" = (", ", ", ")")
            pos1 to (pos2 to pos3)
        }
        .toMap()
}

fun followInstruction(
    instructions: Map<Position, Pair<Position, Position>>,
    currentPosition: Position,
    direction: Direction
): Position {
    val choice = instructions[currentPosition]!!
    return when (direction) {
        Direction.Left -> choice.first
        Direction.Right -> choice.second
    }
}

fun <T> Sequence<T>.repeat() = sequence { while (true) yieldAll(this@repeat) }

fun getPathSequence(
    startingPosition: Position,
    directions: List<Direction>,
    instructions: Map<Position, Pair<Position, Position>>
): Sequence<Position> {
    return directions.asSequence().repeat()
        .scan(startingPosition) { currentPosition, direction ->
            followInstruction(instructions, currentPosition, direction)
        }
}

fun getSteps(input: List<String>): Int {
    val directions = parseDirections(input)
    val instructions = parseInstructions(input)

    return getPathSequence("AAA", directions, instructions)
        .takeWhile { it != "ZZZ" }
        .count()
}

## Example

In [3]:
getSteps(exampleInput)

2

In [4]:
getSteps(
    listOf(
        "LLR",
        "",
        "AAA = (BBB, BBB)",
        "BBB = (AAA, ZZZ)",
        "ZZZ = (ZZZ, ZZZ)",
    )
)

6

## Solution

In [5]:
getSteps(input)

19783

# Part 2

In [6]:
fun getSteps(input: List<String>): Long {
    val directions = parseDirections(input)
    val instructions = parseInstructions(input)

    val startingNodes = instructions.keys.filter { it.endsWith("A") }
    val paths = startingNodes.map { getPathSequence(it, directions, instructions).takeWhile { !it.endsWith("Z") } }
    val pathSizes = paths.map { it.count().toLong() }
    return lcm(pathSizes)
}

fun lcm(a: Long, b: Long) = a * (b / gcd(a, b))
fun lcm(input: List<Long>) = input.reduce { current, i -> lcm(current, i) }

fun gcd(a: Long, b: Long): Long {
    var a = a
    var b = b
    while (b > 0) {
        val temp = b
        b = a % b
        a = temp
    }
    return a
}


## Example

In [7]:
getSteps(
    listOf(
        "LR",
        "",
        "11A = (11B, XXX)",
        "11B = (XXX, 11Z)",
        "11Z = (11B, XXX)",
        "22A = (22B, XXX)",
        "22B = (22C, 22C)",
        "22C = (22Z, 22Z)",
        "22Z = (22B, 22B)",
        "XXX = (XXX, XXX)",
    )
)

6

## Solution

In [8]:
getSteps(input)

9177460370549